In [1]:
## Ipython notebook to open emails and store them in a json file

In [194]:
import pandas as pd
import json
from os import listdir
import os
import re
import pymongo

In [31]:
filepath = './FlaskApp/data/maildir/'

In [33]:
users = listdir(pwd)

In [266]:
import pymongo
## Set up Mongo DB
from pymongo import MongoClient

client = MongoClient()
db = client['email']
collection = db['email']


In [265]:
db.email.remove()

{u'n': 28290, u'ok': 1}

In [ ]:
## Main loop for processing
# for a given user, read in the emails, strip out various metrics, 

DIRECTORIES_TO_PROCESS = ['_sent_mail', 'inbox']

def readFiles(filepath, user, master):
    directories = listdir(filepath + user)
    
    ## Create list of emails
    walkthrough = os.walk(filepath+'/'+user)
    userEmails = []
    for i in walkthrough:
        for j in i[2]:
            email = i[0]+'/'+j

            email_id, parsedText = stripFiles(email)
            
            db.email.insert({email_id:parsedText})

    return master


def stripFiles(email):
    
    with open(email) as f:
        text = f.read()
    text = text.decode(errors='ignore').encode('utf-8', errors='ignore')
    
    email_id = re.sub('\.', '',
                      re.sub('\.Java.*', '', 
                             re.sub('Message-ID: <', '', 
                                    re.findall('Message-ID:.*\n',text)[0]), flags=re.DOTALL))
    
    email_from = re.findall('.*enron\.com', 
                            re.sub('From: ', '', 
                                   re.findall('From:.*\n',text)[0]))
    
    email_to = re.findall('.*enron\.com',
                          re.sub('\t','',
                                 re.sub('To: ', '',
                                        re.findall('To:.*:',text, flags=re.DOTALL)[0])))
    
    ccs = re.findall('Cc:.*\n',text)
    if ccs:
        email_ccs = re.findall('.*enron\.com', re.sub('Cc: ', '', ccs[0]))
    else:
        email_ccs = []
        
    email_subject = [re.sub('Subject: ', '',
                            re.findall('Subject:.*\n',text)[0])]

    email_text = re.sub('X-FileName:.*', '',
                         re.findall('X-FileName:.*',text, flags=re.DOTALL)[0])

    
    output = {'email_from':email_from,
              'email_to':email_to, 
              'email_text':email_text,
              'email_ccs':email_ccs,
              'email_subject':email_subject,
              'full_text':text
             }
    
    return email_id, output



master = {}

users = listdir(filepath)
users = users

for user in users:
    readFiles(filepath, user, master)


In [224]:
c = db.email.find()

In [227]:
c.next()

{u'273948341075855703230': {u'email_ccs': [],
  u'email_from': [u'phillip.allen@enron.com'],
  u'email_subject': [u'Re: Pick your Poison?\r\n'],
  u'email_text': [u'\n\r\nNo can do.  \nAre you in the zone?'],
  u'email_to': [u'chad.landry@enron.com'],
  u'full_text': u'Message-ID: <27394834.1075855703230.JavaMail.evans@thyme>\r\nDate: Wed, 9 Aug 2000 06:36:00 -0700 (PDT)\r\nFrom: phillip.allen@enron.com\r\nTo: chad.landry@enron.com\r\nSubject: Re: Pick your Poison?\r\nMime-Version: 1.0\r\nContent-Type: text/plain; charset=us-ascii\r\nContent-Transfer-Encoding: 7bit\r\nX-From: Phillip K Allen\r\nX-To: Chad Landry\r\nX-cc: \r\nX-bcc: \r\nX-Folder: \\Phillip_Allen_June2001\\Notes Folders\\All documents\r\nX-Origin: Allen-P\r\nX-FileName: pallen.nsf\r\n\r\nNo can do.  \nAre you in the zone?'},
 u'_id': ObjectId('5641309804d8644ed1afedbf')}

In [264]:
id, content = stripFiles('/Users/datascientist/Desktop/abergin/Email/FlaskApp/data/maildir/beck-s/deleted_items/13.')
content

{'email_ccs': [],
 'email_from': [],
 'email_subject': ['NCL Philanthropies\r\n'],
 'email_text': '\n\r\n\nMembership:  Please remember that we have commitments to our  philanthropies even though you may have completed your hours.  We are in  need for people to sign up for: \nKEEP PACE TRANSITION NIGHT \nTuesday,  January 29; 6:30 p.m. - 8:30 p.m. \nCall Jane Price at 281-587-1990  \nAtria Retirement Home - Bingo and Painting, Call Diana Denton for dates  (281-370-6765).\n \n-Claire  Loria',
 'email_to': ['sumpter.teresa@enron.com, stone.pam@enron.com, smithey.linda@enron.com',
  'sinitiere.marynelle@enron.com, shrode.cindy@enron.com',
  'short.suzanne@enron.com, scardello.jackie@enron.com',
  'roberts.gina@enron.com, richmond.trisch@enron.com',
  'reese.lisa@enron.com, ramsey.nancy@enron.com',
  'purser.brenda@enron.com, price.lisa@enron.com, price.jane@enron.com',
  'poullard.marie@enron.com, podraza.judy@enron.com',
  'petrak.janet@enron.com, pearce.becky@enron.com',
  'pauley.sharo

In [237]:
content['email_to']

['sumpter.teresa@enron.com, stone.pam@enron.com, smithey.linda@enron.com']

In [262]:
db.email.insert({id:content})

ObjectId('5641369904d8644ed1b05794')